## extracting from recon3d

In [2]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import mass
import numpy as np
import pandas as pd
import equilibrator_api
from equilibrator_api import ComponentContribution, Q_
import sympy as sym
from cobra import Model, Reaction, Metabolite
import cobra.test
from os.path import join
from mass.util import qcqa
from mass.util import qcqa_model
from cobra import DictList
from mass import (
    MassConfiguration, MassMetabolite, MassModel,
    MassReaction, Simulation, UnitDefinition)
from mass.io.json import save_json_model as save_json_mass_model
from mass.visualization import plot_comparison, plot_time_profile
from mass.visualization import (
    plot_ensemble_phase_portrait, plot_ensemble_time_profile)
mass_config = MassConfiguration()
mass_config.irreversible_Keq = float("inf")
print("MASSpy version: {0}".format(mass.__version__))
from six import iteritems
import matplotlib.pyplot as plt
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)
from cobra.io.json import *
import cobra_dict as c_d
import csv


MASSpy version: 0.1.6


In [3]:
#load Recon3D
model_dir = os.path.abspath("models")
data_dir = os.path.abspath("data")
# model_dir
R3D=load_json_cobra_model(filename=os.path.join(model_dir,"Recon3D.json"))
glycolysis_ppp_tca_etc= load_json_cobra_model(filename=os.path.join(model_dir,"glycolysis_ppp_tca_etc_model.json"))



Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-30


In [4]:
_REQUIRED_REACTION_ATTRIBUTES = [
    "id",
    "name",
    "metabolites",
    "lower_bound",
    "upper_bound",
#     "gene_reaction_rule",
]

_REQUIRED_METABOLITE_ATTRIBUTES = ["id", "name", "charge","formula", "compartment"]
_ORDERED_OPTIONAL_METABOLITE_KEYS = [
   
    "_bound",
    "notes",
    "annotation",
]


In [5]:
from cobra_dict import metabolite_to_dict as metabolite_to_dict
from cobra_dict import reaction_to_dict as reaction_to_dict

In [1]:
etc= [
    'FADH2ETC',
    'NADH2_u10mi',
    'CYOR_u10mi',
    'CYOOm2i',
    'MALtm',
    'AKGMALtm',
    'MDH',
    'ASPTA',
    'ASPGLUm',
    'ASPTAm',

] 


In [13]:
reaction_list=[]
for rid in glycolysis_ppp_tca_etc.reactions:
    if rid.id in etc:
        reaction_list.append(rid.id)
len(reaction_list)

10

In [14]:
metabolite_list=[]
for reaction in reaction_list:
    rxn = glycolysis_ppp_tca_etc.reactions.get_by_id(reaction)
    for met in rxn.metabolites:
        if met.id not in metabolite_list:
            metabolite_list.append(met.id)
len(metabolite_list)

27

In [15]:
met_df=pd.DataFrame()
for met in metabolite_list:
    r3d_met= R3D.metabolites.get_by_id(met)
    m=metabolite_to_dict(r3d_met)
    df_2=pd.DataFrame.from_dict(m,orient='index')
    df_2=df_2.T
    met_df=met_df.append(df_2)

met_df=met_df.set_index('id')
met_df

,name,charge,formula,compartment
id,,,,
h_i,H+,0,H,i
h_m,H+,0,H,m
nad_m,Nicotinamide adenine dinucleotide,-1,C21H26N7O14P2,m
nadh_m,Nicotinamide adenine dinucleotide - reduced,-2,C21H27N7O14P2,m
q10_m,Ubiquinone-10,0,C59H90O4,m
q10h2_m,Ubiquinol-10,0,C59H92O4,m
fad_m,Flavin adenine dinucleotide oxidized,-2,C27H31N9O15P2,m
fadh2_m,Flavin adenine dinucleotide reduced,-2,C27H33N9O15P2,m
ficytC_m,Ferricytochrome c,3,C42H54FeN8O6S2,m


In [16]:
csv_met = os.path.join(data_dir,"etc_met_df.csv")
met_df.to_csv(csv_met)

In [17]:
rxn_df=pd.DataFrame()
reactions_not_in_r3d=[]
for rxn in reaction_list:
    if rxn in R3D.reactions:
        r3d_rxn= R3D.reactions.get_by_id(rxn)
        r=reaction_to_dict(r3d_rxn)
    #     print(r)
        df=pd.DataFrame.from_dict(r,orient='index')
        df=df.T
        rxn_df=rxn_df.append(df)
    else:
        reactions_not_in_r3d.append(rxn)

rxn_df=rxn_df.set_index('id')
rxn_df
reactions_not_in_r3d

[]

In [18]:
csv_rxn = os.path.join(data_dir,"etc_reaction_df.csv")
rxn_df.to_csv(csv_rxn)

In [19]:
json_rxn = os.path.join(data_dir,"etc_reaction_df.json")
rxn_df.to_json(json_rxn
,orient="index"
)   